In [20]:
import praw
import re
import json
from datetime import datetime, timedelta
import time

# Below code mostly generated by ChatGPT

# Set up Reddit API client
reddit = praw.Reddit(
    client_id="Efbot9iva7bBckLwx-Ic-g",
    client_secret="OWxn-YzLwhj0nDMPb7Ai72gQxVJjmg",
    user_agent="prescribed-burning",  
)

# List of subreddits you want to search in
subreddits = ["LosAngeles"]

# List of search terms related to prescribed burns
search_terms = [
    '"prescribed burn"', '"controlled burn"', '"prescribed fire"','"wildfire"',
]

# Combine search terms into a single query
search_query = ' OR '.join(search_terms)

# Initialize the results list
results = []

def matches_title(title, search_terms):
    title_lower = title.lower()
    search_terms = [item.strip('"') for item in search_terms]
    # print(search_terms)
    for term in search_terms:
        # print(term)
        # print(title_lower)
        if str(term) in title_lower:
            # print("success")
            return True
    return False

# Setting up a 12 month time interval 
now = int(time.time())
one_year_ago = int((datetime.now() - timedelta(days=365)).timestamp())

# Search posts in each subreddit
for subreddit in subreddits:
    print(f"Searching in r/{subreddit}...")
    for post in reddit.subreddit(subreddit).search(
        ' OR '.join(search_terms),  # Combine search terms
        limit=800,                  # Set limit as needed               
        time_filter="all"           # Retrieve posts from all time
    ): 
        if (matches_title(post.title,search_terms) and (one_year_ago <= post.created_utc <= now)):
            post_data = {
                "title": post.title,
                "location": str(subreddit),
                "score": post.score,
                "date": datetime.utcfromtimestamp(post.created_utc).strftime('%m-%d-%Y'),
                "url": post.url,
                "num_comments": post.num_comments,
                "replies": [],
            }
        
            # Fetch comments (replace "more comments" and limit to 0 to get all comments)
            post.comments.replace_more(limit=0)
        
            for comment in post.comments.list():  # All comments
                post_data["replies"].append({
                "body": comment.body,
                "date": datetime.utcfromtimestamp(comment.created_utc).strftime('%m-%d-%Y')
                })

            # Add post data to the results list
            results.append(post_data)

# Save results to a file
import json
with open("LosAngelesData.json", "w") as f:
    json.dump(results, f, indent=4)

print("Data saved to reddit_prescribed_burns.json")

Searching in r/LosAngeles...
Data saved to reddit_prescribed_burns.json
